In [1]:
from azureml.core import Workspace, Experiment

# Corrected the below to run within a notebook actually on the studio.
# You guys designed this, so I don't understand why you didn't explain the
# various ways to run this, if it was your intent that I learn it.

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RZT7MUFZ9 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-138618
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-138618


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute

### Reminder to me: compute is always named "ProjectOneAMLcompute"...

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Seems like there are a lot of ways to get this done. Discussion in the lecture would be nice.
# Actually, this needs to be more like Phase 1, where you do some work on a cell and get it to run, then do another lesson.
# After reading the hell out of all this material, you guys could have helped with a mind map.

from azureml.core.runconfig import RunConfiguration



# This part is for me, so I can see what I'm doing.
list_vms = AmlCompute.supported_vmsizes(workspace=ws)



###  uncomment to test to here...PRH this works 2/8
# print(list_vms)

### Here is my solution to specifying the vm_size variable ###
compute_config = RunConfiguration()
compute_config.amlcompute.vm_size = "STANDARD_D1_V2"



# IN CASE I NEED THIS LATER
# from azureml.core.runconfig import ComputeTarget
# Where the hell did I put this? I swear these names are all running together...

from azureml.core.compute_target import ComputeTargetException
compute_name = "ProjectOneAMLcompute" # compute name again
vm_size = 'Standard_D2_V2'
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Using existing compute resource')
except ComputeTargetException:
    print('Creating a new compute instance')
    provisioning_config = AmlCompute.provisioning_config(vm_size = vm_size, max_nodes = 4)

# This exercise kinda proves that implicit code is harder to read. 
# How many learning objectives are you trying to jam in here? Maybe make a list and stick to it.

# The above is a "git 'er done" method. The Microsoft tutorial shows a fairly complicated, but disaster-resistant
# block of code that would find or add a true "cluster"...And yes that actually does include reference to nodes.



# Here's the full monty from the Microsoft tutorial, with my comments and questions:

# compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'cpu-cluster')
# compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
# compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MAX_NODES', 4)

# The following uses CPU for virtual machine, but change "STANDARD_D2_V2" to "STANDARD_NC6" to get GPU.
# vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'STANDARD_D2_V2')

# The following code lines show a check for existing compute resources... Seems like a good idea to me.
# if compute_name in ws.compute_targets:
#     compute_target = ws.compute_targets[compute_name]
#     if compute_target and type(compute_target) is AmlCompute:
#         print('found compute target. just use it. '  + compute_name)
# else:
#     print('creating new compute target...')
#     provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=compute_min_nodes, max_nodes=compute_max_nodes)

# creating the cluster, finally...
# compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

# This part I need to read up later:
# Microsoft tutorial reads, "can poll for a minimum number of nodes and for a specific timeout."
# and, "if no min node count is provided it will use the scale setting for the cluster"
# compute_target.wait_for_completion(show_output=True, mind_node_count=None, timeout_in_minutes=20)

# finally, looks like a best practice would be to add the print (below) line to keep status so we can see what's happening.
# For a more detailed view of current AmlCompute status, use get_status()
# print(compute_target.get_status().serialize())
# note that the above line will only work in conjunction with the rest of this block, see line 38 and/or line 46.

compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'ProjectOneAMLcompute')
print(compute_name)

# Question: this cell requires a workspace to run it. I have to start that workspace, and name it. That name is the output of the print command just above here.
# How does the compute_config = RunConfiguration() work, at the top? What's the name of the rsulting compute?
# Output should be "ProjectOneAMLcompute" See my notes top of this cell.

Using existing compute resource
ProjectOneAMLcompute


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

print("The given code at least runs.")

# Specify parameter sampler
### YOUR CODE HERE ###
ps= RandomParameterSampling(  {
#    "learning_rate": normal(10, 3),
#    "keep_probability": uniform(0.05, 0.1),
#    "batch_size": choice(16, 32, 64, 128),
    "C": uniform(0.01, 2.0),
    "max_iter": choice(50, 75, 100, 150, 200)
    }
)
print("The microsoft docs are accurate enough...")

# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=100, slack_factor= 0.2, slack_amount= 0, delay_evaluation=0)


### The following code block was provided by Udacity ###
if "training" not in os.listdir():
    os.mkdir("./training")





The given code at least runs.
The microsoft docs are accurate enough...


In [ ]:
# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###

# SKLearn and estimators are deprecated, so I tried to use the current objects -- ScriptRunConfig.
# But I can't get it to work, so maybe you can point out what I was doing wrong here...
# the script fails and the MS tutorial calls for me to specify conda dependencies here.
#
# ...that YML file doesn't exist, obviously. I didn't want to mess up the runtime.

### Here's where the "ScriptRunConfig" code begins ###
#from azureml.core import ScriptRunConfig
#from azureml.core import Environment

#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path = './conda_dependencies.yml')

#src = ScriptRunConfig(source_directory='.',
#    script ="train.py",
#    arguments="data",
#    compute_target= "ProjectOneAMLcompute", ## or will it be "compute_name"?
#    environment=sklearn_env)
     ### Here's where the "ScriptRunConfig" ends ###

# The below runs, but has a message about deprecation. I believe I'm suppose to ignore the message. 
# Nice.

est = SKLearn(source_directory = '.',
    compute_target=compute_name,
    entry_script='train.py')

In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###

# I'm using too much time writing all these comments for you guys.
# No more comments. ;-)

hyperdrive_config=HyperDriveConfig(estimator=est,
    hyperparameter_sampling=ps,
        policy=policy,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        max_total_runs=100,
        max_concurrent_runs=4)

# You gotta be careful trusting the helpful IDE stuff in these notebooks! Not too good.

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hd_run = exp.submit(hyperdrive_config)
RunDetails(hd_run).show()
hd_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hd_run.get_best_run_by_primary_metric()
parameters = best_run.get_details()['runDefinition']['arguments']
joblib.dump(parameters, filename = 'outputs/prh_model_best')

# There's a lot of discussion about registering models. 
# I don't think I need that for P1, but it's a deployment function, right?
# Thing is, this is a lot of work going by before you guys give me a first chance for feedback. Too long.

# UNCOMMENT IF i NEED TO REGISTER LATER (There's gotta be a better way to do this!)
# See also LAST CELL IN THIS NOTEBOOK to uncomment the last line there, too!
model = best_run.register_model(model_name = 'prh_model_best', model_path = '.')


In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

# OMG I'm losing my mind! I swear I already did this, and I used a variable def for the path! Aaargh!
# This is definitely all running together!!!

ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [3]:
from train import clean_data
# Why did you leave the next object out? 
# Why isn't this stuff all gathered up in one place, with notes about the architecture?
# I don't see how you keep this stuff straight?
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
training_prh = x_train.join(y_train)
# NAME DEFINED training_prh data set CALL LATER

In [4]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric= 'accuracy',
    training_data= training_prh, #from clean data above
    label_column_name= 'y',
    n_cross_validations= 3)

# Cutting specs to the bare bones to fit the exp timeout.

In [5]:
# Submit your automl run

### YOUR CODE HERE ###
prh_exp = Experiment(ws,'prh_model')
auto_run = prh_exp.submit(automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_30fda75a-57f3-4c49-b6dd-e4be07db074f

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [11]:
# Retrieve and save your best automl model.
import joblib

### YOUR CODE HERE ###
best_run, fitted_model = auto_run.get_output()
joblib.dump(fitted_model, filename = 'outputs/prh_model.joblib')

# Uncomment here for registered model. (Ref from cell above.)
# model = auto_run.register_model(model_name = 'prh_model')

compute_target.delete()